<a href="https://colab.research.google.com/github/SanjaySaatyaki/nlp_with_hf/blob/main/3_transformer_anatomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers; datasets

In [2]:
model_ckpt="bert-base-uncased"

In [3]:
from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Embedding(30522, 768)

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
text = "time files like an arrow"
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[2051, 6764, 2066, 2019, 8612]])

In [7]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds[0][0]

tensor([ 6.0369e-01, -6.4256e-01,  3.3513e-01,  3.3618e-01,  3.8749e-02,
         9.5599e-01, -6.2382e-01,  2.7246e-01, -2.9122e-01,  7.0965e-01,
        -6.4515e-01,  1.2867e+00, -5.1750e-03, -6.2985e-01,  7.9518e-01,
         7.7131e-01, -1.6620e+00, -1.2090e+00, -2.3312e+00, -1.4936e+00,
        -2.1422e+00, -1.9119e+00,  1.5822e+00, -3.1173e-01,  5.1870e-01,
        -5.7714e-01, -1.3912e+00,  1.9027e+00, -5.2370e-02, -8.7727e-02,
         1.2382e+00,  1.0610e+00, -4.1785e-01,  1.3347e+00,  6.4573e-01,
        -1.2103e+00, -1.3815e+00,  1.7385e+00, -5.8997e-01, -6.5711e-02,
         1.3320e+00,  5.4702e-01,  2.4155e-01, -5.3744e-01, -2.2419e-01,
         9.4026e-01,  1.3171e+00, -1.5710e+00,  8.4548e-01,  1.7441e+00,
        -9.9509e-01, -5.9725e-01, -2.9318e+00,  8.3404e-02, -1.0004e-01,
        -6.3038e-01, -4.3406e-01, -9.0429e-01, -1.0602e+00, -1.3502e+00,
         5.9465e-01, -6.3170e-01, -4.8582e-01, -6.2767e-01,  1.8421e+00,
        -1.6867e-01, -1.4707e+00, -1.1505e-02,  1.7

In [8]:
import torch
from math import sqrt

query = key = value = inputs_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
scores.size()

torch.Size([1, 5, 5])

In [9]:
import torch.nn.functional as F

weights = F.softmax(scores, dim=-1)
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

In [10]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

In [11]:
def scaled_dot_product_attention(query, key, value):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  return torch.bmm(weights, value)

In [12]:
class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
    super().__init__()
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)

  def forward(self, hidden_state):
    attn_outputs = scaled_dot_product_attention(
        self.q(hidden_state), self.k(hidden_state), self.v(hidden_state)
    )
    return attn_outputs


In [13]:
class MultiHeadAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    embed_dim = config.hidden_size
    num_heads = config.num_attention_heads
    head_dim = embed_dim // num_heads
    self.heads = nn.ModuleList(
        [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
    )
    self.output_linear = nn.Linear(embed_dim, embed_dim)

  def forward(self, hidden_state):
    x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
    x = self.output_linear(x)
    return x


In [16]:
mutlihead_attn = MultiHeadAttention(config)
attn_output = mutlihead_attn(inputs_embeds)
attn_output.size()

torch.Size([1, 5, 768])